In [1]:
import pandas as pd
import re

In [2]:
### add credentials here
df_loc = df[['tableId', 'rowIndex','content','loc_coord', 'loc_kp', 'loc_mer', 'loc_other', 'loc_tract', 'loc_utm']]
df_loc.head()

,tableId,rowIndex,content,loc_coord,loc_kp,loc_mer,loc_other,loc_tract,loc_utm
0,0114daa6-c048-4081-b4cf-7a9d7461fa44,1,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+020,None,None,None,None
1,0114daa6-c048-4081-b4cf-7a9d7461fa44,2,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+120,None,None,None,None
2,0114daa6-c048-4081-b4cf-7a9d7461fa44,3,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+120,None,None,None,None
3,0114daa6-c048-4081-b4cf-7a9d7461fa44,4,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+220,None,None,None,None
4,0114daa6-c048-4081-b4cf-7a9d7461fa44,5,"[[""Land Use"", ""Condition/ Issues Noted (Rating...",None,KP 0+200 to KP 0+513,None,None,None,None


NTS Type	Column1	NTS Location 	Map Sheet No	Map Sheet letter	Map	Grid	Block	Extra-Info	Final NTS Location <br>
B-027-J/094-I-11	NTS_REGEX3	NTS1	94	I	11	27	J	B	B-027-J/094-I-11 <br>
94I-14-E11	NTS2_REGEX	NTS2	94	I	14	11	E	NA	NA-E-11/94-I-14 <br>
a‑90‑I/94O11	NTS3_REGEX	NTS1	94	O	11	90	I	a	a‑90‑I/94-O-11 <br>
B 025 K/094 I 11	NTS1_2_REGEX	NTS1	94	I	11	25	K	B	B-25-K/94-I-11 <br>
41-H/94-I-9	NTS1_3_REGEX	NTS2	94	I	9	41	H	NA	NA-41-H/94-I-9 <br>
			series_number	map_area	map_sheet	block	series_number	quarter_unit	<br>
            quarter_unit, unit, block, series_number, map_area, map_sheet <br>


In [3]:
LAT_LONG_DD_1_REGEX = r'(\d{1,3}(?:\.\d+))\s*([NWSE](?=[^a-z0-9]|$))'
LAT_LONG_DD_2_REGEX = r'(?i)(lat|long|latitude|longitude)[:\s]+(-?\d{1,3}(?:\.\d+))'
LAT_LONG_DMS_REGEX = r'(\d{1,3}(?:\.\d+)?)[°](\d+(?:\.\d+)?)[\'’](\d+(?:\.\d+)?)[\'\"\’\”]+([NWSE])'
#DLS_REGEX1 = r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?W[^a-zA-Z0-9]?(?P<meridian>[0-9P])M?'
DLS_REGEX3 = r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?(W[^a-zA-Z0-9]?(?P<meridian>[0-9P])M?|\s*)'


#DLS_REGEX2 = r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?'
#DLS_REGEX3 =  r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?(W[^a-zA-Z0-9]?(?P<meridian>([0-9P])M?)|\s*)'
#r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?((W[^a-zA-Z0-9]?(?P<meridian>[0-9P])M?)|\s*)'
#NTS_REGEX = r'([a-dA-D])(?P<delim>[^a-zA-Z0-9])(\d{1,3})(?P=delim)([a-lA-L])[^a-zA-Z0-9](\d{1,3})(?P=delim)([a-pA-P])(?P=delim)(1[1-6]|0?[1-9])'
#NTS_REGEX2 = r'([a-dA-D])(?P<delim>[^a-zA-Z0-9]|\w[ ]{2,}\w)(\d{1,3})(?P=delim)([a-lA-L])(?P<delim2>[^a-zA-Z0-9])(\d{1,3})(?P=delim)([a-pA-P])(?P=delim)(1[1-6]|0?[1-9])'
NTS_REGEX3 = '([a-dA-D])([^a-zA-Z0-9])(\d{1,3})([^a-zA-Z0-9])([a-lA-L])([^a-zA-Z0-9])(\d{1,3})([^a-zA-Z0-9])([a-pA-P])([^a-zA-Z0-9])(1[1-6]|0?[1-9])'
NTS1_2_REGEX = '([a-dA-D])(\d{1,3})([a-lA-L])([^a-zA-Z0-9])(\d{1,3})([a-pA-P])(\d{1,3})'
NTS1_3_REGEX = '(\d{1,3})([^a-zA-Z0-9])([a-lA-L])([^a-zA-Z0-9])(\d{1,3})([^a-zA-Z0-9])([a-pA-P])([^a-zA-Z0-9])(\d{1,3})'
NTS2_REGEX = '(\d{1,3})([a-zA-Z])([^a-zA-Z0-9])(\d{1,3})([^a-zA-Z0-9])([a-zA-Z])(\d{1,3})'
NTS3_REGEX = '([a-z])([^a-zA-Z0-9])(\d{1,3})([^a-zA-Z0-9])([A-Z])([^a-zA-Z0-9])(\d{1,3})([A-Z])(\d{1,3})'

MLV_REGEX = 'MLV\)?(\d+)(?:\+(\d+(?:\.\d+)?)(km|m)?)?'

In [4]:
DLS_REGEX3 = r'(?P<lsd>(?:1[1-6]|0?[1-9])(?=[^a-zA-Z0-9])|(?:NW|NE|SW|SE))?[^a-zA-Z0-9]?(?P<section>\d{1,3})(?P<delim>[^a-zA-Z0-9])(?P<township>\d{1,3})(?P=delim)(?P<range>\d{1,3})\s*[^a-zA-Z0-9]?(W[^a-zA-Z0-9]?(?P<meridian>[0-9P])M?|\s*)'
raw_text = 'P 42+134) NW-8-30-4-W4<s>2</s>ite #42 on Figure 2 014-925-681 (SW 1/4 Sec 14-78-23 W6M)'
raw_text_no_space = re.sub('\s+', '', ' '.join(raw_text))
raw_text_no_space_14 = re.sub("1/4Sec","-", raw_text_no_space)
dls_tags = re.findall(DLS_REGEX3, raw_text_no_space_14)
print(dls_tags)


[('NW', '8', '-', '30', '4', 'W4', '4'), ('', '014', '-', '925', '681', '', ''), ('SW', '14', '-', '78', '23', 'W6M', '6')]


In [29]:
raw_text = '41-H/94-I-9 to 51-H/94-I-9 and A 095 E/094 I 10 to  A 094 E/094 I 10 A 094 E/094 I 10 to  B 093 E/094 I 10'

nts1_2_tags = re.findall(NTS1_2_REGEX, re.sub('\s+', '', ' '.join(raw_text)))
nts1_3_tags = re.findall(NTS1_3_REGEX, re.sub('\s+', '', ' '.join(raw_text)))
print(raw_text)
print(nts1_2_tags)
print(nts1_3_tags)
i =0
for m in re.findall(NTS1_3_REGEX, re.sub('\s+', '', ' '.join(raw_text))):
    i = i+1
    if i == loc_no_local:
        print("abut to check ",m)
        print(m.end())
        if ('to' in raw_text[m.end()-1:m.end()+5]) or (':' in raw_text[m.end()-1:m.end()+5]):
            print("found to")
            nts_tags_temp = re.findall(NTS1_3_REGEX, 
                                       re.sub('\s+', '', ' '.join(raw_text[m.end()-1:])))
            print(nts_tags_temp)

41-H/94-I-9 to 51-H/94-I-9 and A 095 E/094 I 10 to  A 094 E/094 I 10 A 094 E/094 I 10 to  B 093 E/094 I 10
[('A', '095', 'E', '/', '094', 'I', '10'), ('A', '094', 'E', '/', '094', 'I', '10'), ('A', '094', 'E', '/', '094', 'I', '10'), ('B', '093', 'E', '/', '094', 'I', '10')]
[('41', '-', 'H', '/', '94', '-', 'I', '-', '9'), ('51', '-', 'H', '/', '94', '-', 'I', '-', '9')]
abut to check  ('41', '-', 'H', '/', '94', '-', 'I', '-', '9')


AttributeError: 'tuple' object has no attribute 'end'

In [30]:
raw_text = '017-838-967 (NW 1/4 Sec 10-78-23 W6M) 94P-4-J31 94P-4-I40 94P-4-I50 94P-4-I50 94P-4-I49 94P-4-I48 a‑59‑A/94O14 to c‑49‑A/94O14(KP 0.00 to KP 1.00) c‑49‑A/94O14 to a‑40‑A/94O14(KP 1.00 to KP 2.00)  017-838-967 (NW 1/4 Sec 10-78-23 W6M)'
dls_tags = re.findall(DLS_REGEX3, re.sub('\s+', '', ' '.join(raw_text)))
print(re.sub('\s+', '', ' '.join(raw_text)))
print(re.sub("1/4Sec","-",re.sub('\s+', '', ' '.join(raw_text))))
# nts2_tags = re.findall(NTS2_REGEX, raw_text)
# print(nts2_tags)
print("DLS TAGS-- ",dls_tags)
dls_tags = re.findall(DLS_REGEX3, re.sub("1/4Sec","-",re.sub('\s+', '', ' '.join(raw_text))))
print("DLS TAGS-- ",dls_tags)

017-838-967(NW1/4Sec10-78-23W6M)94P-4-J3194P-4-I4094P-4-I5094P-4-I5094P-4-I4994P-4-I48a‑59‑A/94O14toc‑49‑A/94O14(KP0.00toKP1.00)c‑49‑A/94O14toa‑40‑A/94O14(KP1.00toKP2.00)017-838-967(NW1/4Sec10-78-23W6M)
017-838-967(NW-10-78-23W6M)94P-4-J3194P-4-I4094P-4-I5094P-4-I5094P-4-I4994P-4-I48a‑59‑A/94O14toc‑49‑A/94O14(KP0.00toKP1.00)c‑49‑A/94O14toa‑40‑A/94O14(KP1.00toKP2.00)017-838-967(NW-10-78-23W6M)
DLS TAGS--  [('', '017', '-', '838', '967', '', ''), ('', '10', '-', '78', '23', 'W6M', '6'), ('', '017', '-', '838', '967', '', ''), ('', '10', '-', '78', '23', 'W6M', '6')]
DLS TAGS--  [('', '017', '-', '838', '967', '', ''), ('NW', '10', '-', '78', '23', 'W6M', '6'), ('', '017', '-', '838', '967', '', ''), ('NW', '10', '-', '78', '23', 'W6M', '6')]


In [6]:
raw_text = '94P-4-I50 94P-4-I49 94P-4-I48 a‑59‑A/94O14 to c‑49‑A/94O14(KP 0.00 to KP 1.00) c‑49‑A/94O14 to a‑40‑A/94O14(KP 1.00 to KP 2.00) a‑90‑I/94O11 to a‑80‑I/94O11(KP 7.00 to KP 8.00)a-8-L/94-I-14 to c-97-E/94-I-14'
raw_text = 'D 029 J/094 I 11 to  C 028 J/094 I 11'

dls_tags = re.findall(DLS_REGEX3, re.sub('\s+', '', ' '.join(raw_text)))
nts_tags2 = re.findall(NTS_REGEX3, re.sub('\s+', '', ' '.join(raw_text)))
nts2_tags = re.findall(NTS2_REGEX, re.sub('\s+', '', ' '.join(raw_text)))
nts3_tags = re.findall(NTS3_REGEX, re.sub('\s+', '', ' '.join(raw_text)))
nts1_2_tags = re.findall(NTS1_2_REGEX, re.sub('\s+', '', ' '.join(raw_text)))
nts1_3_tags = re.findall(NTS1_3_REGEX, re.sub('\s+', '', ' '.join(raw_text)))

print(raw_text)
print(nts_tags2)
print(nts2_tags)
print(nts3_tags)
print(nts1_2_tags)
print(nts1_3_tags)
loc_no_local = 1

loc_no_local = 0 
i = 0
#storing the matched nts1.2 tags in the table 
for nts_loc in nts1_2_tags:
    print("1. 2nd location found")
    loc_no_local = loc_no_local + 1
    for m in re.finditer(NTS1_2_REGEX, re.sub('\s+', '', ' '.join(raw_text))):
        i = i+1
        print("2. 2nd location found")
        if i == loc_no_local:
            print("3. 2nd location found--")
            print(re.sub('\s+', '', ' '.join(raw_text))[m.end()-1:m.end()+5])
            if ('to' in raw_text[m.end()-1:m.end()+5]) or (':' in raw_text[m.end()-1:m.end()+5]):
                print("4. 2nd location found")
                nts_tags_temp = re.findall(NTS_REGEX3, 
                                        re.sub('\s+', '', ' '.join(raw_text[m.end()-1:])))
                if len(nts_tags_temp) > 0:
                    print("5. 2nd location found")
                    print(nts_tag_temp)

D 029 J/094 I 11 to  C 028 J/094 I 11
[]
[]
[]
[('D', '029', 'J', '/', '094', 'I', '11'), ('C', '028', 'J', '/', '094', 'I', '11')]
[]
1. 2nd location found
2. 2nd location found
3. 2nd location found--
1toC02
2. 2nd location found
1. 2nd location found
2. 2nd location found
2. 2nd location found


In [11]:
for index, row in df_loc.iterrows():
    check = 0
    raw_text = str(row['loc_coord'])
    nts_tags2 = re.findall(NTS_REGEX3, re.sub('\s+', '', ' '.join(raw_text)))
    nts2_tags = re.findall(NTS2_REGEX, re.sub('\s+', '', ' '.join(raw_text)))
    nts2_tags2 = re.findall(NTS2_REGEX, (raw_text))
    if( len(nts_tags2) > 0) or (len(nts2_tags) > 0) or len(nts2_tags2) >0:
        print(raw_text)
        print(nts_tags2)
        print(nts2_tags)
        print(nts2_tags2)
        print("-------New location ------")
    

94P-4-J31
[]
[('94', 'P', '-', '4', '-', 'J', '31')]
[('94', 'P', '-', '4', '-', 'J', '31')]
-------New location ------
94P-4-I40
[]
[('94', 'P', '-', '4', '-', 'I', '40')]
[('94', 'P', '-', '4', '-', 'I', '40')]
-------New location ------
94P-4-I50
[]
[('94', 'P', '-', '4', '-', 'I', '50')]
[('94', 'P', '-', '4', '-', 'I', '50')]
-------New location ------
94P-4-I50
[]
[('94', 'P', '-', '4', '-', 'I', '50')]
[('94', 'P', '-', '4', '-', 'I', '50')]
-------New location ------
94P-4-I50
[]
[('94', 'P', '-', '4', '-', 'I', '50')]
[('94', 'P', '-', '4', '-', 'I', '50')]
-------New location ------
94P-4-I49
[]
[('94', 'P', '-', '4', '-', 'I', '49')]
[('94', 'P', '-', '4', '-', 'I', '49')]
-------New location ------
94P-4-I48
[]
[('94', 'P', '-', '4', '-', 'I', '48')]
[('94', 'P', '-', '4', '-', 'I', '48')]
-------New location ------
94P-4-J74
[]
[('94', 'P', '-', '4', '-', 'J', '74')]
[('94', 'P', '-', '4', '-', 'J', '74')]
-------New location ------
94P-4-J64
[]
[('94', 'P', '-', '4', '-',

C-021-K/094-I-11
[('C', '-', '021', '-', 'K', '/', '094', '-', 'I', '-', '11')]
[]
[]
-------New location ------
D-021-K/094-I-11
[('D', '-', '021', '-', 'K', '/', '094', '-', 'I', '-', '11')]
[]
[]
-------New location ------
D-021-K/094-I-11
[('D', '-', '021', '-', 'K', '/', '094', '-', 'I', '-', '11')]
[]
[]
-------New location ------
C-030-J/094-I-11
[('C', '-', '030', '-', 'J', '/', '094', '-', 'I', '-', '11')]
[]
[]
-------New location ------
D-030-J/094-I-11
[('D', '-', '030', '-', 'J', '/', '094', '-', 'I', '-', '11')]
[]
[]
-------New location ------
D-030-J/094-I-11
[('D', '-', '030', '-', 'J', '/', '094', '-', 'I', '-', '11')]
[]
[]
-------New location ------
C-029-J/094-I-11
[('C', '-', '029', '-', 'J', '/', '094', '-', 'I', '-', '11')]
[]
[]
-------New location ------
C-029-J/094-I-11
[('C', '-', '029', '-', 'J', '/', '094', '-', 'I', '-', '11')]
[]
[]
-------New location ------
C-029-J/094-I-11
[('C', '-', '029', '-', 'J', '/', '094', '-', 'I', '-', '11')]
[]
[]
-------New

In [12]:
table_id = []
row_id = []
loc_no = []
loc_text = []
loc_format = []
start_loc = []
end_loc = []
check_raw_text = []

end_point_loc =0

locations_found = 0
rows_with_locs = 0
other_locations = []
count_nts = 0
count_lat_lang = 0
count_DMS = 0
count_others = 0
for index, row in df_loc.iterrows():
    check = 0
    raw_text = str(row['loc_coord'])
    raw_text_no_space = re.sub('\s+', '', ' '.join(raw_text))
    raw_text_no_space_14 = re.sub("1/4Sec","-", raw_text_no_space)
    
    #dls_tags = re.findall(DLS_REGEX3, re.sub('\s+', '', ' '.join(raw_text)))
    dls_tags = re.findall(DLS_REGEX3, raw_text_no_space_14)
    nts_tags = re.findall(NTS_REGEX3, raw_text_no_space)
    nts1_2_tags = re.findall(NTS1_2_REGEX, raw_text_no_space)
    nts1_3_tags = re.findall(NTS1_3_REGEX, raw_text_no_space)

    nts2_tags = re.findall(NTS2_REGEX, raw_text_no_space)
    nts3_tags = re.findall(NTS3_REGEX, raw_text_no_space)
    
    lat_long_1tags = re.findall(LAT_LONG_DD_1_REGEX, raw_text_no_space)
    lat_long_2tags = re.findall(LAT_LONG_DD_2_REGEX, raw_text_no_space)
    lat_long_DMStags = re.findall(LAT_LONG_DMS_REGEX, raw_text_no_space)
    
    
    loc_no_local = 0
    continue_flag = 0
        
    if len(dls_tags) ==0 and len(nts_tags) == 0 and len(nts1_2_tags) == 0 and len(nts1_3_tags) == 0 and len(nts2_tags)==0 and len(nts3_tags) == 0 and len(raw_text) > 2 and raw_text != 'None':
        other_locations.append(raw_text)
        print(raw_text)
    
    loc_no_local = 0
    #storing the matched nts tags in the table 
    for nts_loc in nts_tags:
        loc_no_local = loc_no_local + 1
    
        if continue_flag == 1:
            continue_flag = 0
            continue
        
        table_id.append(row['tableId'])
        row_id.append(row['rowIndex'])
        
        loc_no.append(loc_no_local)
        
        loc_text.append(row["loc_coord"])
        loc_format.append("NTS")
        
        quarter_unit, unit, block, series_number, map_area, map_sheet = nts_loc[0], int(nts_loc[2]), nts_loc[4], int(nts_loc[6]), nts_loc[8], int(nts_loc[10])
        string_nts = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
        start_loc.append(string_nts)
        
        i = 0
        for m in re.finditer(NTS_REGEX3, raw_text_no_space):
            i = i+1
            if i == loc_no_local:
                
                if ('to' in raw_text_no_space[m.end()-1:m.end()+5]) or (':' in raw_text_no_space[m.end()-1:m.end()+5]):
                    nts_tags_temp = re.findall(NTS_REGEX3, 
                                               raw_text_no_space[m.end()-1:])
                    if len(nts_tags_temp) > 0:
                        end_point_loc = end_point_loc +1
                        continue_flag = 1
                        nts_tag_temp = nts_tags_temp[0]
                        
                        quarter_unit, unit, block, series_number, map_area, map_sheet = nts_tag_temp[0], int(nts_tag_temp[2]), nts_tag_temp[4], int(nts_tag_temp[6]), nts_tag_temp[8], int(nts_tag_temp[10])
                        string_nts = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
                        end_loc.append(string_nts)
                        
        if continue_flag == 0:
            end_loc.append(None)
            
        if check ==0:
            check_raw_text.append(0)
        else:
            check_raw_text.append(1)
            
            
    loc_no_local = 0       
    #storing the matched nts1.2 tags in the table 
    for nts_loc in nts1_2_tags:
        loc_no_local = loc_no_local + 1
    
        if continue_flag == 1:
            continue_flag = 0
            continue
        
        table_id.append(row['tableId'])
        row_id.append(row['rowIndex'])
        
        loc_no.append(loc_no_local)
        
        loc_text.append(row["loc_coord"])
        loc_format.append("NTS")
        
        quarter_unit, unit, block, series_number, map_area, map_sheet = nts_loc[0], int(nts_loc[1]), nts_loc[2], int(nts_loc[4]), nts_loc[5], int(nts_loc[6])
        string_nts = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
        start_loc.append(string_nts)
        
        i = 0
        for m in re.finditer(NTS1_2_REGEX, raw_text_no_space):
            i = i+1
            if i == loc_no_local:
                
                if ('to' in raw_text_no_space[m.end()-1:m.end()+5]) or (':' in raw_text_no_space[m.end()-1:m.end()+5]):
                    nts_tags_temp = re.findall(NTS1_2_REGEX, 
                                               raw_text_no_space[m.end()-1:])
                    if len(nts_tags_temp) > 0:
                        end_point_loc = end_point_loc +1
                        continue_flag = 1
                        nts_tag_temp = nts_tags_temp[0]
                        
                        quarter_unit, unit, block, series_number, map_area, map_sheet = nts_tag_temp[0], int(nts_tag_temp[1]), nts_tag_temp[2], int(nts_tag_temp[4]), nts_tag_temp[5], int(nts_tag_temp[6])
                        string_nts = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
                        end_loc.append(string_nts)
                        
        if continue_flag == 0:
            end_loc.append(None)
            
        if check ==0:
            check_raw_text.append(0)
        else:
            check_raw_text.append(1)
            
    loc_no_local = 0
    #storing the matched nts1.3 tags in the table 
    if len(nts_tags) == 0:
        
        for nts_loc in nts1_3_tags:
            loc_no_local = loc_no_local + 1
    
            if continue_flag == 1:
                continue_flag = 0
                continue
        
            table_id.append(row['tableId'])
            row_id.append(row['rowIndex'])
        
            loc_no.append(loc_no_local)
        
            loc_text.append(row["loc_coord"])
            loc_format.append("NTS2")
        
            quarter_unit, unit, block, series_number, map_area, map_sheet = 'NA', int(nts_loc[0]), nts_loc[2], int(nts_loc[4]), nts_loc[6], int(nts_loc[8])
            string_nts = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
            start_loc.append(string_nts)
        
            
            i = 0
            for m in re.finditer(NTS1_3_REGEX, raw_text_no_space):
                i = i+1
                if i == loc_no_local:
                
                    if ('to' in raw_text_no_space[m.end()-1:m.end()+5]) or (':' in raw_text_no_space[m.end()-1:m.end()+5]):
                        nts_tags_temp = re.findall(NTS1_3_REGEX, 
                                                   raw_text_no_space[m.end()-1:])
                        if len(nts_tags_temp) > 0:
                            end_point_loc = end_point_loc +1
                            continue_flag = 1
                            nts_tag_temp = nts_tags_temp[0]
                        
                            quarter_unit, unit, block, series_number, map_area, map_sheet = 'NA', int(nts_tag_temp[0]), nts_tag_temp[2], int(nts_tag_temp[4]), nts_tag_temp[6], int(nts_tag_temp[8])
                            string_nts = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
                            end_loc.append(string_nts)
                        
            if continue_flag == 0:
                end_loc.append(None)
            
            if check ==0:
                check_raw_text.append(0)
            else:
                check_raw_text.append(1)
            
            
    loc_no_local = 0
    #storing the matched nts2 tags in the table 
    for nts_loc in nts2_tags:
        loc_no_local = loc_no_local + 1
    
        if continue_flag == 1:
            continue_flag = 0
            continue
        
        table_id.append(row['tableId'])
        row_id.append(row['rowIndex'])
        
        loc_no.append(loc_no_local)
        
        loc_text.append(row["loc_coord"])
        loc_format.append("NTS2")
        
        quarter_unit, unit, block, series_number, map_area, map_sheet = 'NA', nts_loc[5], int(nts_loc[6]), int(nts_loc[0]), nts_loc[1], int(nts_loc[3])
        string_nts2 = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
        start_loc.append(string_nts2)
        
        i = 0
        for m in re.finditer(NTS2_REGEX, raw_text_no_space):
            i = i+1
            if i == loc_no_local:
                
                if ('to' in raw_text_no_space[m.end()-1:m.end()+5]) or (':' in raw_text_no_space[m.end()-1:m.end()+5]):
                    nts2_tags_temp = re.findall(NTS2_REGEX, 
                                               raw_text_no_space[m.end()-1:])
                    if len(nts2_tags_temp) > 0:
                        end_point_loc = end_point_loc +1
                        continue_flag = 1
                        nts2_tag_temp = nts2_tags_temp[0]
                        
                        quarter_unit, unit, block, series_number, map_area, map_sheet = 'NA', nts2_tag_temp[5], int(nts2_tag_temp[6]), int(nts2_tag_temp[0]), nts2_tag_temp[1], int(nts2_tag_temp[3])
                        string_nts2 = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
                        end_loc.append(string_nts2)
                        
        if continue_flag == 0:
            end_loc.append(None)
            
        if check ==0:
            check_raw_text.append(0)
        else:
            check_raw_text.append(1)
            
    loc_no_local = 0
    #storing the matched nts3 tags in the table 
    for nts3_loc in nts3_tags:

        loc_no_local = loc_no_local + 1
    
        if continue_flag == 1:
            continue_flag = 0
            continue
        
        table_id.append(row['tableId'])
        row_id.append(row['rowIndex'])
        
        loc_no.append(loc_no_local)
        
        loc_text.append(row["loc_coord"])
        loc_format.append("NTS")
        
        quarter_unit, unit, block, series_number, map_area, map_sheet = nts3_loc[0], int(nts3_loc[2]), nts3_loc[4], int(nts3_loc[6]), nts3_loc[7], int(nts3_loc[8])
        string_nts3 = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
        start_loc.append(string_nts3)
        
        i = 0
        for m in re.finditer(NTS3_REGEX, raw_text_no_space):
            i = i+1
            if i == loc_no_local:
                
                if ('to' in raw_text_no_space[m.end()-1:m.end()+5]) or (':' in raw_text_no_space[m.end()-1:m.end()+5]):
                    nts3_tags_temp = re.findall(NTS3_REGEX, 
                                               raw_text_no_space[m.end()-1:])
                    if len(nts3_tags_temp) > 0:
                        end_point_loc = end_point_loc +1
                        continue_flag = 1
                        nts3_tag_temp = nts3_tags_temp[0]
                        
                        quarter_unit, unit, block, series_number, map_area, map_sheet = nts3_tag_temp[0], int(nts3_tag_temp[2]), nts3_tag_temp[4], int(nts3_tag_temp[6]), nts3_tag_temp[7], int(nts3_tag_temp[8])
                        string_nts3 = '{}-{}-{}/{}-{}-{}'.format(quarter_unit, unit, block, series_number, map_area, map_sheet)
                        end_loc.append(string_nts3)
                        
                        
        if continue_flag == 0:
            end_loc.append(None)
            
        if check ==0:
            check_raw_text.append(0)
        else:
            check_raw_text.append(1)


    loc_no_local = 0
    #storing the matched dls tags in the table 
    for dls_loc in dls_tags:
        loc_no_local = loc_no_local + 1
    
        if continue_flag == 1:
            continue_flag = 0
            continue
        
        
        #exclue cases like 017-838-967
        legal_subdivision, meridian = dls_loc[0], dls_loc[6]
        if legal_subdivision == '' and meridian == '':
            continue
        
        table_id.append(row['tableId'])
        row_id.append(row['rowIndex'])
        
        loc_no.append(loc_no_local)
        
        loc_text.append(row["loc_coord"])
        loc_format.append("DLS")
        
        legal_subdivision, section, township, range_ = dls_loc[0], int(dls_loc[1]), int(dls_loc[3]), int(dls_loc[4])
        meridian = 0
        if dls_loc[6] == '':
            mer = str(row['loc_mer'])
            meridian = mer[1]
        else:
            meridian = dls_loc[6]
        string_dls = '{}-{}-{}-{}-W{}M'.format(legal_subdivision, section, township, range_, meridian)
        start_loc.append(string_dls)
        
        if len(meridian) == 0 or len(legal_subdivision) !=2:
            check = 1
        
        i = 0
        for m in re.finditer(DLS_REGEX3, raw_text_no_space_14):
            i = i+1
            if i == loc_no_local:
                
                if ('to' in raw_text_no_space_14[m.end()-1:m.end()+5]) or (':' in raw_text_no_space_14[m.end()-1:m.end()+5]):
                    dls_tags_temp = re.findall(DLS_REGEX3, raw_text_no_space_14[m.end()-1:])
                    if len(dls_tags_temp) > 0:
                        
                        end_point_loc = end_point_loc +1
                        continue_flag = 1
                        dls_tag_temp = dls_tags_temp[0]
                        
                        legal_subdivision, section, township, range_ = dls_tag_temp[0], int(dls_tag_temp[1]), int(dls_tag_temp[3]), int(dls_tag_temp[4])
                        
                        meridian = 0
                        if dls_tag_temp[6] == '':
                            mer = str(row['loc_mer'])
                            meridian = mer[1]
                        else:
                            meridian = dls_tag_temp[6]
                        string_dls = '{}-{}-{}-{}-W{}M'.format(legal_subdivision, section, township, range_, meridian)
                        end_loc.append(string_dls)
                        
                        if meridian == 0 or len(legal_subdivision) != 2:
                            check = 1
        
        if continue_flag == 0:
            end_loc.append(None)
            
        if check ==0:
            check_raw_text.append(0)
        else:
            check_raw_text.append(1)


<s>--</s>
<s>--</s>
<s>--</s>
<s>--</s>
<s>--</s>
<s>--</s>
<s>--</s>
<s>--</s>
<s>--</s>
<s>--</s>
<s>--</s>
<s>--</s>
Moderate slopes
Pipeline Loop
Pipeline Loop
Pipeline Loop
Pipeline Loop - Rycroft Soils
Pipeline Loop - Rycroft Soils
Pipeline Loop - Rycroft Soils
Pipeline Loop
Same as above
Pipeline Loop
Pipeline Loop
Unnamed Creek at KP 9.2
Unnamed Creek at KP 9.2
Pipeline Loop
Pipeline Loop
Pipeline Loop


In [13]:
df_locations_extracted = pd.DataFrame({'tableId': table_id, 
                                      'rowIndex': row_id, 
                                      'locNo' : loc_no, 
                                      'locText': loc_text,
                                      'locFormat': loc_format,
                                      'startLoc': start_loc,
                                      'endLoc' : end_loc,
                                      'checkRawText': check_raw_text})
                                       
df_locations_extracted.to_csv('pcmr_locations_dls_n_nts.csv')